In [3]:
from external_models_mofa import *
import sys
sys.path.append('../')

from importlib.metadata import version

from copy import copy

In [2]:
import muon as mu
import anndata as ad
import mofax
import mofapy2

/home/gosia/miniconda3/envs/cling_mofa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(sys.version)

3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]


In [5]:
print(mu.__version__)
print(ad.__version__)
print(version('mofax'))
print(mofapy2.__version__)

0.1.7
0.12.2
0.3.7
0.7.2


In [6]:
N = 100
M = 4
D = [20 for m in range(M)]
K = 5

In [7]:
def generate_simple(K, N, M, D, noise=0.5):
    z = np.random.normal(size = (N, K))
    w = [np.random.normal(size = (K, D)) for m in range(M)]
    data = {'M'+str(m): np.dot(z, w[m]) + np.random.normal(size = (N,D), scale=noise) for m in range(M)}

    return data, z, w

In [9]:
data, z, w = generate_simple(K, N, M, D[0])

## MOFA without pruning

In [10]:
mod = model_mofa(data, 10, seed=123, explained_variance_treshold=None)
mod.fit()


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Features names not provided, using default naming convention:
- feature1_view1, featureD_viewM

Groups names not provided, using default naming convention:
- group1, group2, ..., groupG

Samples names not provided, using default naming convention:
- sample1_group1, sample2_group1, sample1_group2, ..., sampleN_groupG

Successfully loaded view='M0' group='group0' with N=100 sample

In [11]:
model = mod.get_model()

In [12]:
mod.get_factors()[:5, :]

array([[ 1.50162938e+000, -1.28021741e+000,  4.60115538e-001,
        -7.08737534e-001, -1.03192601e+000, -9.71301844e-175,
        -1.44055162e-175,  6.18432164e-177,  2.54623671e-176,
         2.34599405e-177],
       [ 4.81854373e-001, -4.71588814e-001, -1.35911297e+000,
        -2.22634927e+000, -1.54381734e+000,  1.83068894e-175,
         2.77738281e-176, -1.19174248e-177, -4.85270210e-177,
        -4.50046814e-178],
       [ 1.77355542e+000, -1.47426426e+000,  4.55876676e-001,
        -8.93366738e-001,  7.38300930e-001,  9.56986510e-175,
         1.41860292e-175, -6.11056457e-177, -2.51111000e-176,
        -2.31190627e-177],
       [-1.25861083e+000,  1.13358470e+000, -1.61051233e+000,
        -1.52272608e+000, -1.87668406e+000,  1.17583683e-174,
         1.74910813e-175, -7.50847931e-177, -3.09748191e-176,
        -2.85097214e-177],
       [-3.84138079e-001,  7.42843470e-001, -3.88195451e-001,
         1.02033541e+000,  3.06108671e-001,  1.32948673e-174,
         1.97460630e-175

In [13]:
mod.get_weights()[0][:5,:]

array([[ 3.28729966e-001, -1.18145346e+000, -3.52356593e-001,
         1.48657071e-001, -6.54834038e-001, -4.42901709e-176,
        -6.56073001e-177,  2.82947469e-178,  1.16178895e-177,
         1.06941271e-178],
       [-7.07134818e-002,  8.36478381e-003, -5.45655588e-001,
        -8.41980111e-002,  4.80107218e-001,  3.48065057e-176,
         5.14693218e-177, -2.21896686e-178, -9.10513639e-178,
        -8.38562144e-179],
       [-3.71007828e-001,  8.04433505e-001, -1.42395256e+000,
        -1.39816716e+000, -2.98535352e-001,  2.03989490e-176,
         3.02923439e-177, -1.30435423e-178, -5.35575230e-178,
        -4.93412311e-179],
       [-2.95931526e-001,  8.13650214e-001,  1.02624975e+000,
         5.49387659e-001,  2.30219536e-002, -2.17396189e-176,
        -3.24497385e-177,  1.40033051e-178,  5.72385288e-178,
         5.28053598e-179],
       [-1.06641594e+000, -1.19541107e+000,  3.39400184e-001,
         2.44987976e-002,  9.46010314e-001, -1.34574342e-176,
        -2.00494803e-177

In [14]:
mod.variance_explained_per_factor_per_view()

array([[0.21862929, 0.36620172, 0.38029991, 0.3902524 ],
       [0.27988232, 0.25122493, 0.27361717, 0.24518304],
       [0.24473512, 0.13427303, 0.17637743, 0.11247989],
       [0.16254714, 0.13032763, 0.08206556, 0.1366784 ],
       [0.05753214, 0.075489  , 0.05355439, 0.06236514],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

In [15]:
mod2 = copy(mod.get_model())
np.array(mod2.model.calculate_variance_explained()[0]).T

array([[0.21862929, 0.36620172, 0.38029991, 0.3902524 ],
       [0.27988232, 0.25122493, 0.27361717, 0.24518304],
       [0.24473512, 0.13427303, 0.17637743, 0.11247989],
       [0.16254714, 0.13032763, 0.08206556, 0.1366784 ],
       [0.05753214, 0.075489  , 0.05355439, 0.06236514],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

In [16]:
mod.variance_explained_per_factor()

array([0.34174623, 0.26226276, 0.16552907, 0.12662213, 0.06230043,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [17]:
mod.variance_explained_per_view()

array([0.95829102, 0.9638775 , 0.96169826, 0.96024722])

In [18]:
np.array(mod2.model.calculate_variance_explained(total=True)[0])

array([0.95829102, 0.9638775 , 0.96169826, 0.96024722])

In [19]:
mod.get_number_of_active_factors()

10

# MOFA with pruning after the training

In [20]:
mod = model_mofa(data, 10, seed=123, explained_variance_treshold=0.12)
mod.fit()


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Features names not provided, using default naming convention:
- feature1_view1, featureD_viewM

Groups names not provided, using default naming convention:
- group1, group2, ..., groupG

Samples names not provided, using default naming convention:
- sample1_group1, sample2_group1, sample1_group2, ..., sampleN_groupG

Successfully loaded view='M0' group='group0' with N=100 sample

In [21]:
mod.K_fitted

4

# MOFA with pruning during the training

In [22]:
mod = model_mofa_prune(data, 10, seed=123, explained_variance_treshold=0.12)
mod.fit()


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Features names not provided, using default naming convention:
- feature1_view1, featureD_viewM

Groups names not provided, using default naming convention:
- group1, group2, ..., groupG

Samples names not provided, using default naming convention:
- sample1_group1, sample2_group1, sample1_group2, ..., sampleN_groupG

Successfully loaded view='M0' group='group0' with N=100 sample

In [23]:
mod.K_fitted

3